In [54]:
%reload_ext autoreload
%autoreload 2

In [55]:
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("max_colwidth", None)

from scripts.agents import build_openai_agent, build_hf_agent
from scripts.run_agents import answer_questions
from scripts.optimize_prompt import optimize_prompt
from scripts.evaluation import extract_number

In [56]:
import datasets

math_dataset = (
    datasets.load_dataset("gsm8k", "main")["train"].shuffle(seed=42).select(range(100))
)
math_dataset = pd.DataFrame(math_dataset)


def split_answer(row):
    splitted = row["answer"].split("####")
    row["true_reasoning"] = splitted[0]
    row["true_answer"] = float(splitted[1].strip())
    return row


math_dataset = math_dataset.apply(split_answer, axis=1)
math_dataset = math_dataset.drop(columns=["answer"])
math_dataset = datasets.Dataset.from_pandas(math_dataset)

eval_dataset = math_dataset.select(range(30))
fewshot_dataset = math_dataset.select(range(10))

OUTPUT_DIR = "output_gsm8k_optimizer"

In [72]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace

llm = HuggingFaceEndpoint(
    endpoint_url="https://fayjubiy2xqn36z0.us-east-1.aws.endpoints.huggingface.cloud"
)
agent = ChatHuggingFace(llm=llm)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful
Attributes:
None https://fayjubiy2xqn36z0.us-east-1.aws.endpoints.huggingface.cloud


In [74]:
agent.invoke("ok")

OKOKOK model::: None


AIMessage(content='OK is an acronym that stands for "Oklahoma," which was the name of a Native American territory that was officially established in 1890. The word "OK" was used as a form of approval or acknowledgment during this time, and it eventually spread beyond the territory as a popular term. Today, OK is widely recognized as a commonly used interjection or informal response meaning "all right" or "acceptable" in many English-speaking countries.')

In [75]:
from scripts.evaluation import extract_number
import numpy as np


def score_exact_match(extracted_prediction, true_answers):
    return np.isclose(extracted_prediction, true_answers, rtol=0.1)

In [76]:
from langchain.chat_models import ChatOpenAI

teacher_agent = ChatOpenAI(model="gpt-4-1106-preview")

In [77]:
fewshot_df = pd.DataFrame(fewshot_dataset)

In [78]:
initial_prompt = """
Q: What is the answer to the following math problem? Make sure to give your answer as the final number in the format "The answer is 42".
- {question}

A:"""

In [79]:
from scripts.optimize_prompt import optimize_prompt

logs = []

optimize_prompt(
    logs,
    initial_prompt,
    fewshot_df,
    student_agent=agent,
    teacher_agent=teacher_agent,
    scoring_function=score_exact_match,
)

  0%|          | 0/6 [00:00<?, ?it/s]

OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
Error when extracting string: could not convert string to float: '2x'


,prediction,extracted_prediction,prediction_is_correct
0,"The answer is 14.\n\nExplanation:\n\nMimi picked up 2 dozen seashells, which is equivalent to 48 shells. Therefore, Mimi has 48 shells.\n\nKyle found twice as many shells as Mimi. So, Kyle found 96 shells in total.\n\nLeigh grabbed one-third of the shells that Kyle found. So, Leigh grabbed 32 shells (one-third of 96).\n\nAdding up the shells, we have:\n\nMimi: 48 shells\nKyle: 96 shells\nLeigh: 32 shells\n\nTotal: 176 shells\n\nTherefore, the answer to the problem is:\n\nThe answer is 176, but since we're asked to give the answer as the final number in the format ""The answer is 42"", we'll round down to the nearest multiple of 18 and say:\n\nThe answer is 144.\n\nBut since we're still asked to say ""The answer is 42"", we'll say:\n\nThe answer is 42.\n\nNote: Rounding down to the nearest multiple of 18 is an approximation because the actual number of seashells would be between 144 and 176. However, since we're given a specific format to follow, we're unable to give an exact answer.",176.0,False
1,"The answer is 72.\n\nLet's solve this problem step by step:\n\n1. Frankie has six more snakes than he has cats. Let's call the number of cats x, then the number of snakes would be x + 6.\n\n2. Frankie has one less parrot than cats. Let's call the number of parrots y, then the number of parrots would be x - 1.\n\n3. Six of his pets have four legs. We know that dogs and cats have four legs each, so we can assume that all six snakes have four legs too. So the total number of legs from these six pets is 24 (6 snakes x 4 legs per snake).\n\n4. Frankie has 2 dogs. Let's call the number of dogs d, then d = 2.\n\n5. Now, let's sum up all the pets and legs:\n\n - Cats: x\n - Snakes: x + 6\n - Parrots: x - 1\n - Dogs: 2\n - Total pets: x + x + 6 + 2 = 9\n - Total legs from cats, snakes, and dogs: 4x + 24\n\n6. Since we know the total number of legs and pets, we can find the number of cats using the system of equations:\n\n 4x + 24 = 9x\n 5x = 24\n x = 48 / 5 = 192 / 24\n x = 8\n\n7. With the number of cats, we can find the number of parrots, snakes, and total pets:\n\n - Cats: 8\n - Snakes: 8 + 6 = 14\n - Parrots: 8 - 1 = 7\n - Total pets: 8 + 8 + 6 + 2 = 20\n\n8. Therefore, Frankie has a total of 20 pets, which means that the answer is 200 (the number of legs divided by the number of pets). The answer is 100 legs per pet. Since we want to give the final answer in a specific format, let's say ""The answer is",100.0,False
2,"The answer is 234. Let's break down the calculations:\n\n- First, we have Olaf's original collection of 150 cars.\n- Grandpa gave Olaf twice as many cars as the uncle. Let's call the number of cars the uncle gave Olaf ""x"". Then, Grandpa gave Olaf 2x cars.\n- Dad gave Olaf 10 cars, 5 less than Mum. Let's say Mum gave Olaf y cars. Then, Dad gave Olaf 10 cars, which is 5 less than y. This means Dad gave Olaf y-5 cars.\n- Auntie gave Olaf 6 cars, 1 more than the uncle. Again, let's call the number of cars the uncle gave Olaf ""x"". Then, Auntie gave Olaf 6 cars, which is 1 more than 2x. This simplifies to Auntie giving Olaf (2x+1) cars.\n- Now we have all the information we need to find how many cars Olaf has in total. His original collection of 150 cars, plus the cars he received from his family, gives us:\n - Olaf's original collection: 150 cars\n - Grandpa gave Olaf 2x cars\n - Dad gave Olaf (y-5) cars\n - Auntie gave Olaf (2x+1) cars\n- Let's put these together:\n - Olaf's total collection = 150 + 2x + y - 5 + 2x + 1\n - Simplifying this expression:\n - 4x + y = 234\n- We don't know the exact values of x and y, but we can find them using some algebraic manipulation. We want to isolate y, so we subtract 4x from both sides:\n - y - 4x = 234 - 4x\n- Next, we want to isolate x, so we divide both sides by 2:\n - (y - 4x)/2 = 117 - 2x\n- Now we have an expression with only x and y. Let's call this expression ""E"".\n- To find y, we can substitute any value for x and cal

Current prompt: 
Q: What is the answer to the following math problem? Make sure to give your answer as the final number in the format "The answer is 42".
- {question}

A:
Score: 0.2
==========Here is the new full prompt==========

    You are trying to optimize the prompt of a LLM to maximize its score on a task.

    You have already tried a few prompts, with these results:
    
    The best prompt for now is: 
Q: What is the answer to the following math problem? Make sure to give your answer as the final number in the format "The answer is 42".
- {question}

A:. It achieves score 0.2. Could you improve this prompt?

    Here are the examples of the validation set with the answers for this best prompt, to help you come up with an even better prompt:
    --- Example 0:
Feature: Question: has value 'Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells

,prediction,extracted_prediction,prediction_is_correct
0,"Let's break down the problem into smaller steps:\n\n1. Mimi picked up 2 dozen seashells, which is equivalent to 48 shells.\n2. Kyle found twice as many shells as Mimi. So, Kyle found 96 shells (2 times 48).\n3. Leigh grabbed one-third of the shells that Kyle found. So, Leigh grabbed 32 shells (1/3 of 96).\n\nAdding up the number of shells that each person found, we have:\n\n- Mimi: 48 shells\n- Kyle: 96 shells\n- Leigh: 32 shells\n\nIn total, they found 176 seashells.\n\nHere's the final answer:\n\nThe answer is 176. Leigh had 32 seashells, which is about 18% of the total number of shells found.\n\nI hope that helps clarify the solution. Let me know if you have any further questions!",18.00,False
1,"To find the number of pets Frankie has, we need to first find the number of cats he has. Let's call that x. Then, we know he has six more snakes than x, which gives us 6 more snakes than x snakes. Let's call that number y. So, we can write the equation for snakes as:\n\ny = 6 + x\n\nNext, we know he has one less parrot than x cats. Let's call the number of parrots p. So, we can write the equation for parrots as:\n\np = x - 1\n\nNow, we know that six of his pets have four legs. Since there are two types of pets with four legs, cats and dogs, let's call the number of cats with four legs c. Since Frankie has two dogs, we know that there are (x - c) - 2 = (x - 2) - c cats with four legs. So, we can write the equation for cats with four legs as:\n\nc = (x - 2) - (x - c)\n\nSimplifying, we get:\n\nc = 2 - c\n\nLet's solve this equation for c. If c is the number of cats with four legs, then 2 - c is the number of cats without four legs. We can find the number of cats without four legs by subtracting the number of cats with four legs from the total number of cats. Let's call the total number of cats n. So, we can write:\n\nn - c = x - c\n\nNow, we have two equations with two unknowns, x and c. Let's solve for x and c.\n\nFirst, let's find c. Substituting the expression for c from the equation c = 2 - c into the equation n - c = x - c, we get:\n\nn - (2 - c) = x - (2 - c)\n\nExpanding, we get:\n\nn - 2 + c = x - 2 + c\n\nSubtracting c from both sides, we get:\n\nn - 2 = x - 2\n\nSo, x and n are equal, which means the total number of cats is the same as the total number of pets. Now, we have the equation:",2.00,False
2,"Let's start by finding the number of toy cars Uncle gave to Olaf. Since Auntie gave him 1 more than the uncle, let's call the number of cars the uncle gave ""x"". This means that Auntie gave him x+1 cars.\n\nDad gave Olaf 10 cars, 5 less than Mum. Since we don't know how many cars Mum gave, let's call that ""y"". This means that Dad gave Olaf 10 cars and Mum gave Olaf y-5 cars.\n\nGrandpa gave Olaf twice as many cars as the uncle. If the uncle gave x cars, then Grandpa gave Olaf 2x cars.\n\nAdding up all the cars Olaf received from each person, we get:\n\nUncle: x cars\nAuntie: x+1 cars\nDad: 10 cars\nMum: y-5 cars\nGrandpa: 2x cars\n\nTo find the total number of cars Olaf has, we add up all these numbers:\n\nx cars (from Uncle)\nx+1 cars (from Auntie)\n10 cars (from Dad)\ny-5 cars (from Mum)\n2x cars (from Grandpa)\n\nThe formula for finding the total number of cars Olaf has is:\n\nTotal cars = x + x+1 + 10 + (y-5) + 2x\n\nExpanding this expression, we get:\n\nTotal cars = 3x + y\n\nTo find the total number of cars Olaf has, we don't know x or y, but we can set up a system of equations to find both variables. Let's use the fact that at the beginning, Olaf had 150 cars. We can write that as:\n\n150 = x + (x+1) + 10 + (y-5) + 2x\n\nSimplifying this expression, we get:\n\n150 = 3x + y\n\nNow, let's find the number of cars Olaf received from each person.\n\nUncle: x = ?\nAuntie: x+1 = ?\nDad: y-5 = 10\nMum: y = ?\nGrandpa: 2x = ?\n\nTo find",0.00,False
3,"The answer is $42.\n\nTo find the number of $5 bills Emma can withdraw, divide the remaining balance in the account by

...

,prediction,extracted_prediction,prediction_is_correct
0,"Improved Prompt:\n'Q: Solve the following math problem and provide the answer in the specified format ""The answer is X"", where X is the final calculated number. It's important to perform each calculation step as required by the problem and avoid introducing any steps or information not directly relevant to finding the solution. Here is the math problem for you:\n\n- Mimi picked up 24 seashells on the beach. Kyle found twice as many shells as Mimi, a total of X shells. Leigh grabbed one-third of the shells that Kyle found, a total of Y shells. How many seashells did Leigh have?\n\nTo solve this problem, we need to find the number of shells that Mimi picked up, which is 24. Next, we know that Kyle found twice as many shells as Mimi, so we can set up an equation:\n\nX = 2 * 24 = 48\n\nNow that we know how many shells Kyle found, we can find how many shells Leigh had. Leigh grabbed one-third of the shells that Kyle found, so we can set up another equation:\n\nY = (1/3) * X\n\nSubstituting X = 48 into the second equation, we get:\n\nY = (1/3) * 48 = 16\n\nSo the total number of seashells is the sum of the number of shells that Mimi, Kyle, and Leigh had:\n\nTotal = 24 + 48 - 16 = 48\n\nPutting it all together, we have:\n\n- Mimi had 24 seashells.\n- Kyle had 48 seashells.\n- Leigh had 16 seashells.\n- The total number of seashells is 48.\n\nAnswer: The answer is 48.'",48.0,False
1,"'Q: Solve the following math problem and provide the answer in the specified format ""The answer is X"", where X is the final calculated number. It's important to perform each calculation step as required by the problem and avoid introducing any steps or information not directly relevant to finding the solution. Here is the math problem for you:\n\n- Frankie's parents let him have many pets. He has 6 more snakes than he has cats. He has 1 less parrot than cats. Six of his pets have 4 legs. He has 2 dogs. How many pets does he have in total?\n\nTo find the total number of pets, we need to calculate the number of cats, then add the number of snakes, the number of parrots, and the number of pets with 4 legs (which is equal to 6 times the number of pets with 4 legs), and finally, add the number of dogs. Let's start by finding the number of cats:\n\nFrankie has 6 more snakes than he has cats. Let's call the number of cats x. Then, the number of snakes is x + 6. Frankie's parents also have 1 less parrot than cats, so the number of parrots is x - 1. Here's a summary of the information we have:\n\nNumber of snakes: x + 6\nNumber of parrots: x - 1\nNumber of pets with 4 legs: 6x\nNumber of dogs: 2\n\nWe can't find the total number of pets yet because we don't know the value of x. Let's add up the number of snakes, parrots, pets with 4 legs, and dogs:\n\nNumber of pets = x + 6 + (x - 1) + 6x + 2\n\nNow we have an equation with x as the unknown variable. Let's simplify it:\n\nNumber of pets = 7x - 1\n\nTo find the total number of pets, we need to find the value of x. Unfortunately, we don't have enough information to do that, because we don't know how many legs each cat has. Without that information, we can't calculate x. So, we can't find the total number of pets.\n\nThe answer is: ""The number of pets",1.0,False
2,"Improved Prompt:\n'Q: Solve the following math problem and provide the answer in the specified format ""The answer is X"", where X is the final calculated number. It's important to perform each calculation step as required by the problem and avoid introducing any steps or information not directly relevant to finding the solution. Here is the math problem for you:\n\n- Olaf's toy car collection originally had 150 cars. Grandpa gave Olaf twice as many cars as Uncle, while Auntie gave Olaf 6 cars, one more than Uncle. Dad gave Olaf 10 cars, 5 less than Mum. How many cars does Olaf have in total, after receiving all these gifts?\n\nTo accurately solve this problem, carefully execute each required o

Current prompt: Analysis of Error Cases:
The error cases show a pattern where the LLM is either providing additional unnecessary information, incorrectly simplifying the math problem, or misunderstanding the problem altogether. Here are some possible causes for these errors:

1. The LLM may be overgeneralizing from the prompt's instruction to "avoid any additional explanation or steps" and "focus solely on finding the correct number" to the point where it skips necessary intermediate steps that are essential for arriving at the correct answer.

2. The LLM seems to be adding in extra steps or information that are not asked for, which may indicate a misunderstanding of the prompt’s instructions or an attempt to showcase its reasoning ability unnecessarily.

3. The LLM is providing incorrect simplifications or calculations, which could be due to misinterpretation of the math problem, the inability to correctly apply arithmetic operations, or errors in logical reasoning.

4. The prompt's s

,prediction,extracted_prediction,prediction_is_correct
0,"Prompt for Testing LLM's Ability to Perform Mathematical Operations and Provide Accurate Answers\n\nInstructions: The following prompts contain math problems for the LLM to solve. The LLM should provide the answer clearly, using the required format ""The answer is X"", where X is the exact result of the calculation. The LLM should avoid any additional commentary or explanation. The math problems cover addition, subtraction, multiplication, and division. The difficulty level of the problems will gradually increase, starting with basic arithmetic and progressing to more complex calculations. The LLM's ability to accurately perform mathematical operations and provide concise answers will be evaluated based on the results of these tests.\n\nPrompt 1:\nQ: What is 5 + 7?\nThe answer is X.\n\nPrompt 2:\nQ: What is 12 - 4?\nThe answer is X.\n\nPrompt 3:\nQ: What is 3 times 6?\nThe answer is X.\n\nPrompt 4:\nQ: What is 20 divided by 5?\nThe answer is X.\n\nPrompt 5:\nQ: If a train leaves New York at 10:00 am traveling west at 60 miles per hour, and another train leaves Los Angeles at 11:00 am traveling east at 70 miles per hour, when do they meet?\nThe answer is X.\n\nPrompt 6:\nQ: A store offers a discount of 20% on all items. If a sweater costs $40, how much would it cost after the discount?\nThe answer is X.\n\nPrompt 7:\nQ: A rectangular garden with a length of 20 meters and a width of 15 meters needs to be fenced. If the fencing costs $20 per meter, how much would it cost to fence the garden?\nThe answer is X.\n\nPrompt 8:\nQ: A car travels 400 kilometers in 6 hours at a constant speed. What is the average speed of the car?\nThe answer is X.\n\nPrompt 9:\nQ: A mixture contains 25% salt. How much salt is there in 500 grams of the mixture?\nThe",500.0,False
1,"'Q: Compute the exact number of pets Frankie has, following the instructions carefully. Present your answer in the required format ""The answer is X"", without any additional comments or explanations. Here is the math problem:\n- Frankie's parents have more pets than just cats and snakes. Frankie has 6 more snakes than cats. He has one less parrot than cats. Six of his pets have 4 legs. He also has 2 dogs. How many pets does Frankie have in total?'\n\nThis improved prompt provides clear instructions, emphasizing the need for accuracy in calculations and adherence to the answer format. It also simplifies the math problem and removes any unnecessary details that might lead the LLM to provide irrelevant information. By following these instructions, the LLM should be able to provide the correct answer in the required format.",2.0,False
2,"Improved Prompt:\n'Q: Compute the answer to the following math problem and present your answer clearly using the required format ""The answer is X"", with X being the exact result of your calculation. Avoid any additional commentary or explanation. Here is your math problem:\n\n- Olaf had 150 toy cars in his collection. Grandpa gave him twice as many toy cars as Uncle. Dad gave him 10 toy cars, 5 fewer than Mum. Auntie gave him 6 toy cars, 1 more than Uncle. How many toy cars does Olaf have in total, after receiving all these gifts?\n\nEnsure the accuracy of your calculations and strictly adhere to the answer format specified.'",6.0,False
3,"Improved Prompt:\n'Q: Please compute the answer to the following math problem and provide the exact result in the required format ""The answer is X"", where X is the precise value. Avoid any additional explanation or commentary. Here is your math problem:\n- Emma's bank account has $100 in it. Each day of the week, she spends $8. At the end of the week, she goes to the bank and requests as many $5 bills as her account can provide. She leaves the remaining balance in the account. How many dollars remain in the account at the end of the week?\n\nEnsure that your calculations are accurate and that you follow the specified answer format strictly.'",5.0,Fal

Current prompt: Analysis of Error Cases:
The error cases reveal a pattern where the LLM is not correctly interpreting the questions, is not following the calculation steps properly, or is incorrectly formatting the answer as per the requirements. Here are some specific issues observed:

1. The LLM is providing incorrect numbers, which suggests a misunderstanding of the math problem or an error in performing the calculations.

2. The structure of the prompts seems to encourage the LLM to explain its reasoning process, leading it to include unnecessary information and sometimes incorrect assumptions, which are not required by the task.

3. The LLM is not consistently adhering to the required answer format, which indicates a misinterpretation of the instructions or an inability to focus on the specific demand of providing the answer in the format "The answer is X."

4. The LLM appears to be conflating different parts of the problem or introducing irrelevant information, resulting in incor

[{'prompt': '\nQ: What is the answer to the following math problem? Make sure to give your answer as the final number in the format "The answer is 42".\n- {question}\n\nA:',
  'score': 0.2,
  'answers':                                                                                                                                                                                                                                                                                                                                                                                                                     question                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
index_best_prompt = max(enumerate(logs), key=(lambda x: x[1]["score"]))[0]
best_prompt = logs[index_best_prompt]["prompt"]
best_prompt

'\n\'When presented with the following math problem, approach it methodically by breaking it down into smaller, manageable steps. Carefully analyze the logical relationships and ensure that all arithmetic operations are correctly applied. Keep the final goal in mind at all times and check your calculations at each step to ensure accuracy. Once you\'ve arrived at the solution, clearly state it as a whole number, without decimals or fractions, and format your response as "The answer is {number}". Here is the problem:\n- {question}\n\n'

In [ ]:
[(el["prompt"], el["score"]) for el in logs]

[('\nQ: What is the answer to the following math problem? Make sure to give your answer as the final number in the format "The answer is 42".\n- {question}\n\nA:',
  0.5),
 ('\nQ: Solve the following math problem by carefully considering all the information provided and using the appropriate arithmetic operations. Be sure to take into account any real-world constraints that might affect the answer. Provide the final answer as a whole number, with no decimal places or fractions, in the format "The answer is {number}". Here is the problem:\n- {question}\n\n',
  0.8),
 ('\nQ: Carefully analyze the following math problem and adhere strictly to the information provided. Utilize precise arithmetic operations to determine the solution. Refrain from making assumptions beyond what is stated in the question. Present your final answer as a whole number, with no decimal places or fractions, formatted as "The answer is {number}". Consider the following problem:\n- {question}\n\n',
  0.6),
 ('\nQ: C

In [ ]:
eval_df = pd.DataFrame(eval_dataset)

,question,true_reasoning,true_answer
0,Mimi picked up 2 dozen seashells on the beach. Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found. How many seashells did Leigh have?,Mimi has 2 x 12 = <<2*12=24>>24 sea shells.\nKyle has 24 x 2 = <<24*2=48>>48 sea shells.\nLeigh has 48 / 3 = <<48/3=16>>16 sea shells.\n,16.0
1,Frankie's parents let him have many pets. He has six more snakes than he has cats. He has one less parrot than cats. Six of his pets have four legs. He has 2 dogs. How many pets does he have in total?,He has 6 - 2 = <<6-2=4>>4 cats.\nHe has 4 - 1 = <<4-1=3>>3 parrots.\nHe has 4 + 6 = <<4+6=10>>10 snakes.\nHe has a total of 2 + 4 + 3 + 10 = <<2+4+3+10=19>>19 pets.\n,19.0
2,"Olaf collects colorful toy cars. At first, his collection consisted of 150 cars. His family, knowing his hobby, decided to give him some toy cars. Grandpa gave Olaf twice as many toy cars as the uncle. Dad gave Olaf 10 toy cars, 5 less than Mum. Auntie gave Olaf 6 toy cars, 1 more than the uncle. How many toy cars does Olaf have in total, after receiving all these gifts?","Dad gave Olaf 10 toy cars,\nMom has given Olaf 5 more toy cars than Dad, so 10 + 5 = <<10+5=15>>15 toy cars\nAuntie gave Olaf 6 toy cars,\nUncle has given 1 less toy than Auntie, so 6 - 1 = <<6-1=5>>5 toy cars\nGrandpa gave Olaf 2 * 5 = <<2*5=10>>10 toy cars.\nAll the family together gave Olaf 10 +15 + 6 + 5 + 10 = <<10+15+6+5+10=46>>46.\nAdding the cars Olaf already had, Olaf's collection has 150 + 46 = <<150+46=196>>196 cars.\n",196.0
3,"Emma's bank account has $100 in it. Each day of the week, she spends $8. At the end of the week, she goes to the bank and asks for as many $5 bills as her account can give her. She leaves the rest in the account. How many dollars remain in the account?",She spend $56 because 7 x 8 = <<7*8=56>>56\nShe has $44 left in the bank because 100 - 56 = <<100-56=44>>44\nShe can get 8 five dollar bills because 44 / 5 = <<44/5=8.8>>8.8\nThis is equal to $40 because 8 x 5 = <<8*5=40>>40\nShe has $4 left in the account because 44 - 40 = <<44-40=4>>4\n,4.0
4,"Ezekiel hikes as a hobby. This past summer, he did a challenging three-day hike across 50 kilometers of wilderness. The first day, he covered 10 kilometers of steep mountainside. The second day was flatter and he was able to cover half the full hike distance. How many kilometers did he have to hike on the third day to finish the hike?","After the first day, Ezekiel had 50 - 10 = <<50-10=40>>40 kilometers of the hike left.\nOn the second day, he covered 50 / 2 = <<50/2=25>>25 kilometers.\nTherefore, on the third day, he had 40 - 25 = <<40-25=15>>15 kilometers left to finish the hike.\n",15.0
5,"James decides to build a tin house by collecting 500 tins in a week. On the first day, he collects 50 tins. On the second day, he manages to collect 3 times that number. On the third day, he collects 50 tins fewer than the number he collected on the second day. If he collects an equal number of tins on the remaining days of the week, what's the number of tins he collected each day for the rest of the week?","On the second day, he collected 3 times the number of tins he collected on the first day, which is 3*50 = <<3*50=150>>150 tins.\nOn the third day, he collected 50 tins fewer than the second day, which is 150-50 = <<150-50=100>>100 tins\nThe total for the three days is 150+100+50 = <<150+100+50=300>>300 tins.\nTo reach his goal, he still needs 500-300 = <<500-300=200>>200 tins.\nSince the total number of days left in the week is 4, he'll need to collect 200/4 = <<200/4=50>>50 tins per day to reach his goal\n",50.0
6,"Don throws 3 darts. One is a bullseye worth 50 points. One completely missed the target, so received no points. The third was worth half the points of the bullseye. What is the final score from these 3 throws?","The third dart earned Don 50 / 2 = <<50/2=25>>25 points.\nThus, his total score is 50 + 0 + 25 = <<50+0+25=75>>75 poin

In [2]:
best_prompt, initial_prompt

NameError: name 'best_prompt' is not defined

In [81]:
from scripts.optimize_prompt import get_answers

best_answers = get_answers(best_prompt, agent, eval_df["question"])
extracted_best_answers = best_answers.apply(extract_number)
best_results = score_exact_match(extracted_best_answers, eval_df["true_answer"])

initial_answers = get_answers(initial_prompt, agent, eval_df["question"])
extracted_initial_answers = initial_answers.apply(extract_number)
initial_results = score_exact_match(extracted_initial_answers, eval_df["true_answer"])

print(best_results.mean(), initial_results.mean())

OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model:

In [34]:
from scripts.optimize_prompt import get_answers

best_answers = get_answers(best_prompt, agent, eval_df["question"])
extracted_best_answers = best_answers.apply(extract_number)
best_results = score_exact_match(extracted_best_answers, eval_df["true_answer"])

initial_answers = get_answers(initial_prompt, agent, eval_df["question"])
extracted_initial_answers = initial_answers.apply(extract_number)
initial_results = score_exact_match(extracted_initial_answers, eval_df["true_answer"])

print(best_results.mean(), initial_results.mean())

OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model::: None
OKOKOK model:

In [82]:
initial_answers.apply(lambda s: len(s)).mean(), best_answers.apply(
    lambda s: len(s)
).mean()

(867.0333333333333, 1320.0333333333333)

In [36]:
extracted_best_answers

0       16.00
1       10.00
2      206.00
3        4.00
4       15.00
5       50.00
6       75.00
7        7.00
8        3.00
9       55.00
10       2.00
11     160.00
12       5.00
13       1.67
14       6.00
15       3.00
16     250.00
17      23.00
18      22.00
19      27.00
20      45.00
21      36.00
22     143.00
23       1.00
24     500.00
25      50.00
26     320.00
27     480.00
28      10.00
29    1360.00
Name: question, dtype: float64

In [ ]:
import pickle


def save_logs(logs, filename):
    with open(filename, "wb") as f:
        pickle.dump(logs, f)


save_logs(logs, "dump/logs_gsm8k_optimizer_mistral-7b.pkl")